In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import StratifiedKFold

# Load pheno

In [2]:
pheno_p = '/Users/harveyaa/Documents/masters/data/pheno_26-01-22.csv'
pheno = pd.read_csv(pheno_p, index_col=0)

/var/folders/5x/9dwn0lbx633gccy22ts5bskh0000gn/T/ipykernel_4943/1011432404.py:2: DtypeWarning: Columns (7,8,12,13,14,19,20,24,27,31,42,43,44,49,51,59,60,62,64,65,68,101,121,163) have mixed types. Specify dtype option on import or set low_memory=False.
  pheno = pd.read_csv(pheno_p, index_col=0)


# Load datasets

In [3]:
#cases = ['ADHD','ASD','BIP','SZ','DEL1q21_1','DUP1q21_1','DEL22q11_2','DUP22q11_2','DEL16p11_2','DUP16p11_2']
cases = ['DEL15q11_2']
id_p = '/Users/harveyaa/Documents/masters/neuropsych_mtl/datasets/{}.txt'
ids = []
for case in cases:
    ids.append(pd.read_csv(id_p.format(case),header=None))
ids = dict(zip(cases,ids))

# Get CV splits

In [4]:
cvs = []
for case in cases:
    print(case)
    df = pheno[pheno.index.isin(ids[case][0])]

    skf = StratifiedKFold(n_splits=5,shuffle=True)
    res = skf.split(df.index,df['SITE'])

    cols = []
    for _,r in res:
        col = np.zeros(len(df.index))
        col[r] = 1
        cols.append(col)
    splits = pd.DataFrame(cols,columns=df.index, index=['fold_0','fold_1','fold_2','fold_3','fold_4']).transpose()
    splits.to_csv(f'{case}.csv')
    cvs.append(splits)

DEL15q11_2


# Investigate splits

In [5]:
ratios = []
for i,case in enumerate(cases):
    cv = cvs[i]

    ratio = []
    for fold in [0,1,2,3,4]:
        subs = cv[cv[f'fold_{fold}']==1].index
        ratio.append(pheno[pheno.index.isin(subs)][case].mean())
    ratios.append(ratio)

In [6]:
df = pd.DataFrame(ratios,index=cases)
df['mean'] = df.mean(axis=1)

df

,0,1,2,3,4,mean
DEL15q11_2,0.571429,0.487805,0.560976,0.439024,0.439024,0.499652
